# Past Values and Past Errors: The ARMA Model

* Limitations of AR or MA models 

  * Both AR and MA have residuals that is not white noise. This means neither of them is suitable for fitting price return data. 
  * AR limitation: fail to adjust quickly to unexpected shocks --> need MA aspects to smooth out prediction
  * MA limitation:  need a baseline to perform well --> need to use previous values (AR) 
  * AR + MA: solves the issues each one has individually 

* Definition 

  * Equation ARMA(1) 
    $$
    r_t = c + \varphi_1r_{t-1} + \theta_1\epsilon_{t-1} + \epsilon_t
    $$

    * $r_t$, $r_{t-1}$: The values of "r" in the current period and 1 period ago respectively
    * $\epsilon_t$, $\epsilon_{t-1}$ : Residuals for the current period and the 1 period ago respectively
    * $c$: Baseline constant factor 
    * $\varphi_1$: The part of the value last period is relevant in explaining the current one, $|\varphi_n|<1$ to prevent compounded effects exploding in magnitude
    * $\theta_1$: The part of the value last period is relevant in explaining the current one, $|\theta_n|<1$ to prevent compounded effects exploding in magnitude 

  * ARMA(p, q)

    * p: number of lagged values (AR order)
    * q: number of lagged errors (MA order)
    * p and q don't have to be the same value  
    * Actual - prediction -> how foar off our predictions were -> calibrate expections on the go 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima_model import ARMA
from scipy.stats.distributions import chi2
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv('../data/Index2018.csv')
df_comp = raw_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst=True)
df_comp.set_index("date", inplace=True)
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')

In [3]:
df_comp['market_value'] = df_comp.ftse
del df_comp['spx']
del df_comp['dax']
del df_comp['ftse']
del df_comp['nikkei']

size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size].copy(), df_comp.iloc[size:].copy()

In [4]:
df['returns'] = df.market_value.pct_change(1).mul(100)

In [5]:
def LLR_test(mod_1, mod_2, DF=1):
    # DF: degrees of freedom
    
    # log likelihood
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    
    # test statistic
    LR = (2 * (L2 - L1))
    p = chi2.sf(LR, DF).round(3)
    
    return p

# ARMA(1, 1)

In [6]:
model_ret_ar_1_ma_1 = ARMA(df.returns[1:], order=(1,1))
results_ret_ar_1_ma_1 = model_ret_ar_1_ma_1.fit()
results_ret_ar_1_ma_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                     ARMA(1, 1)   Log Likelihood               -7916.500
Method:                       css-mle   S.D. of innovations              1.171
Date:                Tue, 26 Jan 2021   AIC                          15841.000
Time:                        23:42:57   BIC                          15867.085
Sample:                    01-10-1994   HQIC                         15850.140
                         - 04-05-2013                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0189      0.013      1.446      0.148      -0.007       0.045
ar.L1.returns     0.7649      0.067     11.349      0.000       0.633       0.897
ma.L1.returns    -0.8141      0.061    -13.406      0.000      -0.933      -0.695
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.3074           +0.0000j            1.3074            0.0000
MA.1            1.2284           +0.0000j            1.2284            0.0000
-----------------------------------------------------------------------------
"""

The p-value for constant is not significantly different from zero. As we are using returns, it is within reason for returns to be centered around zero. This is completely acceptible. 

The coefficient for past value is over 0.5 meaning there is a positive tendency between past and present values. In other words, returns move in trends of consecutive positive or negative. When translating it into prices, prices tends to consistently increase or decrease. 

The negative coefficient for the past error is slightly harder to interpret. We should be moving away from the past period (t-1) values. These pas error terms ensure we don't get a "fool in the shower" type of error.   

We want to compare the ARMA(1, 1) with the AR(1) and MA(1) models using LLR test because they are nested in the ARMA(1, 1).

In [8]:
# LLR Test 
model_ret_ar_1 = ARMA(df.returns[1:], order=(1,0))
model_ret_ma_1 = ARMA(df.returns[1:], order=(0,1))


print("ARMA vs AR", LLR_test(model_ret_ar_1, model_ret_ar_1_ma_1))
print("ARMA vs MA", LLR_test(model_ret_ma_1, model_ret_ar_1_ma_1))

ARMA vs AR 0.0
ARMA vs MA 0.0


This verifies that ARMA(1,1) is better than AR(1) and MA(1).